# WEB SCRAPER AO - AIRLINEQUALITY

In [2]:
# Libraries

import bs4
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
from functools import reduce
import numpy as np

# Sources

Air Inequality
https://www.airlinequality.com/

In [39]:
# URL 
ae_url = 'https://www.airlinequality.com/'
ae_url_airfrance = 'https://www.airlinequality.com/airline-reviews/air-france/'

# Test Get Reviews

In [40]:
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(ae_url_airfrance,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page)


l_titles = []

bodies = soup.body

bodies = soup.find_all("div", {"class":"body"})

## Get general value

In [41]:
company = soup.find('div', {'class':'info'})
company_name = company.h1.text.strip()

l_global_value_header = []
l_global_value_stars = []

stats = soup.table
tdTags = stats.find_all("td")
for tag in tdTags:
    if 'header' in tag['class'][0]:
        l_global_value_header.append(tag.text)
    if 'stars' in tag['class'][0]:
        tag_values = tag.find_all('span', {'class':'star fill'})
        l_global_value_stars.append(int(tag_values[-1].text))

l_global = [('company_name', company_name)] + list(zip(l_global_value_header, l_global_value_stars))

In [42]:
pages = soup.find('article', {'class':'comp comp_reviews-pagination querylist-pagination position-'})
l_pages = pages.find_all('li')
max_page = int(l_pages[-2].text.strip())
max_page

102

## Get Reviews values

In [43]:
l_titles = []
l_reviewer_data = []
l_verifications = []
l_reviews = []
l_stats = []
for i in range(len(bodies)):
    title = bodies[i].h2.text
    l_titles.append(title)

    reviewer_data = bodies[i].h3.text #to be proccessed
    l_reviewer_data.append(reviewer_data)

    review = bodies[i].find("div", {'class':'text_content'})
    verification = review.a.text
    l_verifications.append(verification)
    review = review.text #to be proccessed 
    l_reviews.append(review)

    l_review_value = []
    l_review_value_header = []
    l_review_value_stars = []

    stats = bodies[i].table
    tdTags = stats.find_all("td")
    #print(tdTags)
    for tag in tdTags:
        #print(tag['class'][0], tag)
        if 'header' in tag['class'][0]:
            l_review_value_header.append(tag.text)
        if tag['class'][0] == 'review-value':
            l_review_value.append(tag.text)
        if 'stars' in tag['class'][0]:
            tag_values = tag.find_all('span', {'class':'star fill'})
            l_review_value_stars.append(int(tag_values[-1].text))
    l_review_value[-1:-1] = l_review_value_stars
    
    l_stats.append(list(zip(l_review_value_header, l_review_value)))

print(len(l_titles) == len(bodies))
print(len(l_reviewer_data) == len(bodies))
print(len(l_verifications) == len(bodies))
print(len(l_reviews)== len(bodies))
print(len(l_stats) == len(bodies))

True
True
True
True
True


In [44]:
#Transform our data to dataframes
dfs = [pd.DataFrame(l) for l in l_stats]

#Merge accoring to column 0
df = reduce(lambda left,right: pd.merge(left,right,on=0, how='outer'), dfs)

#Get the right format of dataframe
df_reviews = df.T.copy()
df_reviews.columns = df_reviews.iloc[0]
df_reviews.drop(df_reviews.index[0], inplace=True)
df_reviews.reset_index(drop=True, inplace=True)

In [45]:
df_reviews.head()

,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value For Money,Recommended,Aircraft,Wifi & Connectivity
0,Solo Leisure,Business Class,Toronto to Paris,January 2020,5,5,1,4,5,2,no,NaN,NaN
1,Solo Leisure,Economy Class,Prague to Cape Town via Paris,January 2020,1,5,1,NaN,1,1,no,NaN,NaN
2,Solo Leisure,Economy Class,Toronto to Paris,December 2019,5,1,1,2,5,1,no,NaN,NaN
3,Business,Business Class,Paris to Chicago,January 2020,4,4,4,4,2,4,yes,A330-300,3
4,Solo Leisure,Business Class,Paris to Madrid,January 2020,1,2,1,1,3,1,no,A320,1


# Functions

In [46]:
def scraping_airlinequality(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)

    page_soup = BeautifulSoup(page, "html.parser")

    company = soup.find('div', {'class':'info'})
    company_name = company.h1.text.strip()

    l_global_value_header = []
    l_global_value_stars = []

    stats = soup.table
    tdTags = stats.find_all("td")
    for tag in tdTags:
        if 'header' in tag['class'][0]:
            l_global_value_header.append('global ' + tag.text)
        if 'stars' in tag['class'][0]:
            tag_values = tag.find_all('span', {'class':'star fill'})
            l_global_value_stars.append(int(tag_values[-1].text))

    l_global = [('company_name', company_name)] + list(zip(l_global_value_header, l_global_value_stars))
    
    bodies = soup.body

    bodies = soup.find_all("div", {"class":"body"})
    
    l_df = [] #a list that stores data of the url
    
    for i in range(len(bodies)):
        
        l_data_url = [] #a list that stores data of the review of the url (zip format)
        l_data_url += l_global #we add global values for each reviews
        
        title = bodies[i].h2.text
        title = title.strip('"')
        l_data_url.append(('title', title))
        
        reviewer_data = bodies[i].h3.text.strip() 
        reviewer_data = reviewer_data.split('\n')[-1]
        
        date = ' '.join(reviewer_data.split()[-3:]) #the date is the last words of reviewer data
        country = reviewer_data.split('(')[-1].split(')')[0] #we split according to '(' and ')'
        
        l_data_url.append(('date', date))
        l_data_url.append(('country', country))
        
        review = bodies[i].find("div", {'class':'text_content'})
        verification = review.a
        if verification is None:
            l_data_url.append(('verification', np.nan))
        else:
            l_data_url.append(('verification', verification.text))
        
        review_content = review.text
        review_content = review_content.split('| ')[-1]
        l_data_url.append(('review', review_content))
        
        l_review_value = []
        l_review_value_header = []
        l_review_value_stars = []

        stats = bodies[i].table
        tdTags = stats.find_all("td")
        for tag in tdTags:
            if 'header' in tag['class'][0]:
                l_review_value_header.append(tag.text)
            if tag['class'][0] == 'review-value':
                l_review_value.append(tag.text)
            if 'stars' in tag['class'][0]:
                tag_values = tag.find_all('span', {'class':'star fill'})
                if len(tag_values) != 0:
                    l_review_value_stars.append(int(tag_values[-1].text))
        l_review_value[-1:-1] = l_review_value_stars

        l_data_url += list(zip(l_review_value_header, l_review_value))
        l_df.append(l_data_url)
        
        #Transform our data to dataframes
    dfs = [pd.DataFrame(l) for l in l_df]

    #Merge accoring to column 0
    df = reduce(lambda left,right: pd.merge(left,right,on=0, how='outer'), dfs)

    #Get the right format of dataframe
    df_reviews = df.T.copy()
    df_reviews.columns = df_reviews.iloc[0]
    df_reviews.drop(df_reviews.index[0], inplace=True)
    df_reviews.reset_index(drop=True, inplace=True)
    
    return df_reviews

# Running code

Scrapping, it's better to launch it on google colab I think

In [ ]:
scraped_companies = ['air-france',
 'american-airlines',
 'air-china',
 'lufthansa',
 'emirates',
 'ana-all-nippon-airways',
 'latam-airlines',
 'aeroflot-russian-airlines',
 'air-canada',
 'singapore-airlines']

In [ ]:
results_dfs = []
for companie in scraped_companies:
    global_url = 'https://www.airlinequality.com/airline-reviews/{}/'.format(str(companie))
    print(global_url)
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(global_url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)
    pages = soup.find('article', {'class':'comp comp_reviews-pagination querylist-pagination position-'})
    l_pages = pages.find_all('li')
    max_page = int(l_pages[-2].text.strip())
    print(max_page)
    for n_page in range(max_page) :
        company_url = global_url + 'page/{}'
        url = company_url.format(str(n_page))
        print(url)
        df = scraping_airlinequality(url)
        results_dfs.append(df)
df_reviews = pd.concat(results_dfs, axis=0)

In [9]:
df_reviews.shape

(10, 13)

# I want to scrape all the airline companies

In [47]:
# a function to return a list of airlines
def all_airlines():
    url = 'https://www.airlinequality.com/review-pages/a-z-airline-reviews/'
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)
    section = soup.find('div', id='a2z-ldr-A')
    airlines = section.findAll('a',href=True)
    airlines = [a.get_text() for a in airlines]
    
    return airlines


In [54]:
scraped_companies = all_airlines()

In [57]:
failures = []
successes = []
for companie in scraped_companies:
    try:
        companie = companie.replace(' ','-')

        results_dfs = []
        global_url = 'https://www.airlinequality.com/airline-reviews/{}/'.format(str(companie))
        print(global_url)
        hdr = {'User-Agent': 'Mozilla/5.0'}
        req = Request(global_url,headers=hdr)
        page = urlopen(req)
        soup = BeautifulSoup(page)
        pages = soup.find('article', {'class':'comp comp_reviews-pagination querylist-pagination position-'})
        l_pages = pages.find_all('li')
        max_page = int(l_pages[-2].text.strip())
        print(max_page)
        for n_page in range(max_page) :
            company_url = global_url + 'page/{}'
            url = company_url.format(str(n_page))
            print(url)
            df = scraping_airlinequality(url)
            results_dfs.append(df)
        df_reviews = pd.concat(results_dfs, axis=0)

        # Save files to the disk
        directory = 'AirlineQuality_data/' + companie + '.csv'
        df_reviews.to_csv(directory)
        successes.append(companie)
    except:
        failures.append(companie)

https://www.airlinequality.com/airline-reviews/AB-Aviation/
https://www.airlinequality.com/airline-reviews/Adria-Airways/
10
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/0
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/1
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/2
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/3
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/4
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/5
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/6
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/7
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/8
https://www.airlinequality.com/airline-reviews/Adria-Airways/page/9


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



https://www.airlinequality.com/airline-reviews/Aegean-Airlines/
58
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/0
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/1
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/2
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/3
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/4
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/5
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/6
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/7
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/8
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/9
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/10
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/11
https://www.airlinequality.com/airline-reviews/Aegean-Airlines/page/12
https://www.airlineq

https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/61
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/62
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/63
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/64
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/65
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/66
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/67
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/68
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/69
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/70
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/71
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/72
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/73
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/74
https://www.airlinequality.com/airline-reviews/Aer-Lingus/page/75
https://ww

https://www.airlinequality.com/airline-reviews/Aeromexico/page/9
https://www.airlinequality.com/airline-reviews/Aeromexico/page/10
https://www.airlinequality.com/airline-reviews/Aeromexico/page/11
https://www.airlinequality.com/airline-reviews/Aeromexico/page/12
https://www.airlinequality.com/airline-reviews/Aeromexico/page/13
https://www.airlinequality.com/airline-reviews/Aeromexico/page/14
https://www.airlinequality.com/airline-reviews/Aeromexico/page/15
https://www.airlinequality.com/airline-reviews/Aeromexico/page/16
https://www.airlinequality.com/airline-reviews/Aeromexico/page/17
https://www.airlinequality.com/airline-reviews/Aeromexico/page/18
https://www.airlinequality.com/airline-reviews/Aeromexico/page/19
https://www.airlinequality.com/airline-reviews/Aeromexico/page/20
https://www.airlinequality.com/airline-reviews/Aeromexico/page/21
https://www.airlinequality.com/airline-reviews/Aeromexico/page/22
https://www.airlinequality.com/airline-reviews/Aeromexico/page/23
https://www

...

https://www.airlinequality.com/airline-reviews/American-Airlines/page/36
https://www.airlinequality.com/airline-reviews/American-Airlines/page/37
https://www.airlinequality.com/airline-reviews/American-Airlines/page/38
https://www.airlinequality.com/airline-reviews/American-Airlines/page/39
https://www.airlinequality.com/airline-reviews/American-Airlines/page/40
https://www.airlinequality.com/airline-reviews/American-Airlines/page/41
https://www.airlinequality.com/airline-reviews/American-Airlines/page/42
https://www.airlinequality.com/airline-reviews/American-Airlines/page/43
https://www.airlinequality.com/airline-reviews/American-Airlines/page/44
https://www.airlinequality.com/airline-reviews/American-Airlines/page/45
https://www.airlinequality.com/airline-reviews/American-Airlines/page/46
https://www.airlinequality.com/airline-reviews/American-Airlines/page/47
https://www.airlinequality.com/airline-reviews/American-Airlines/page/48
https://www.airlinequality.com/airline-reviews/Amer

https://www.airlinequality.com/airline-reviews/American-Airlines/page/148
https://www.airlinequality.com/airline-reviews/American-Airlines/page/149
https://www.airlinequality.com/airline-reviews/American-Airlines/page/150
https://www.airlinequality.com/airline-reviews/American-Airlines/page/151
https://www.airlinequality.com/airline-reviews/American-Airlines/page/152
https://www.airlinequality.com/airline-reviews/American-Airlines/page/153
https://www.airlinequality.com/airline-reviews/American-Airlines/page/154
https://www.airlinequality.com/airline-reviews/American-Airlines/page/155
https://www.airlinequality.com/airline-reviews/American-Airlines/page/156
https://www.airlinequality.com/airline-reviews/American-Airlines/page/157
https://www.airlinequality.com/airline-reviews/American-Airlines/page/158
https://www.airlinequality.com/airline-reviews/American-Airlines/page/159
https://www.airlinequality.com/airline-reviews/American-Airlines/page/160
https://www.airlinequality.com/airline

https://www.airlinequality.com/airline-reviews/American-Airlines/page/259
https://www.airlinequality.com/airline-reviews/American-Airlines/page/260
https://www.airlinequality.com/airline-reviews/American-Airlines/page/261
https://www.airlinequality.com/airline-reviews/American-Airlines/page/262
https://www.airlinequality.com/airline-reviews/American-Airlines/page/263
https://www.airlinequality.com/airline-reviews/American-Airlines/page/264
https://www.airlinequality.com/airline-reviews/American-Airlines/page/265
https://www.airlinequality.com/airline-reviews/American-Airlines/page/266
https://www.airlinequality.com/airline-reviews/American-Airlines/page/267
https://www.airlinequality.com/airline-reviews/American-Airlines/page/268
https://www.airlinequality.com/airline-reviews/American-Airlines/page/269
https://www.airlinequality.com/airline-reviews/American-Airlines/page/270
https://www.airlinequality.com/airline-reviews/American-Airlines/page/271
https://www.airlinequality.com/airline

https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/2
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/3
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/4
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/5
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/6
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/7
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/8
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/9
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/10
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/11
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/12
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/page/13
https://www.airlinequality.com/airline-reviews/ANA-All-Nippon-Airways/pa

https://www.airlinequality.com/airline-reviews/ASKY-Airlines/
2
https://www.airlinequality.com/airline-reviews/ASKY-Airlines/page/0
https://www.airlinequality.com/airline-reviews/ATA-Airlines/
https://www.airlinequality.com/airline-reviews/Atlantic-Airways/
https://www.airlinequality.com/airline-reviews/Atlasglobal/
https://www.airlinequality.com/airline-reviews/Aurigny-Air/
6
https://www.airlinequality.com/airline-reviews/Aurigny-Air/page/0
https://www.airlinequality.com/airline-reviews/Aurigny-Air/page/1
https://www.airlinequality.com/airline-reviews/Aurigny-Air/page/2
https://www.airlinequality.com/airline-reviews/Aurigny-Air/page/3
https://www.airlinequality.com/airline-reviews/Aurigny-Air/page/4
https://www.airlinequality.com/airline-reviews/Aurigny-Air/page/5
https://www.airlinequality.com/airline-reviews/Austrian-Airlines/
56
https://www.airlinequality.com/airline-reviews/Austrian-Airlines/page/0
https://www.airlinequality.com/airline-reviews/Austrian-Airlines/page/1
https://www

In [4]:
# Merge all the data in the directory
import glob
path = 'AirlineQuality_data/'
all_files = glob.glob(path + "/*.csv")

li=[]
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)
df.to_csv('data_airlinequality.csv')

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.
